<img src="https://github.com/pmservice/ai-openscale-tutorials/raw/master/notebooks/images/banner.png" align="left" alt="banner">

# Tutorial on generating an explanation for an image-based model on Watson OpenScale

This notebook includes steps for creating an image-based watson-machine-learning model, creating a subscription, configuring explainability, and finally generating an explanation for a transaction.

### Contents
- [1. Setup](#setup)
- [2. Creating and deploying an image-based model](#deployment)
- [3. Subscriptions](#subscription)
- [4. Explainability](#explainability)

**Note**: If using Watson Studio, try running the notebook on atleast 'Default Python 3.5 XS' version for faster results.

<a id="setup"></a>
## 1. Setup

### 1.1 Install Watson OpenScale and WML packages

In [ ]:
!pip install --upgrade ibm-ai-openscale --no-cache | tail -n 1

In [ ]:
!pip install --upgrade watson-machine-learning-client --no-cache | tail -n 1

Note: Restart the kernel to assure the new libraries are being used.

### 1.2 Configure credentials

Get the Watson Openscale `apikey` by going to the [Bluemix console](https://console.bluemix.net/) and clicking `Manage->Account->Users`. Select `Platform API Keys` from the sidebar and then click the "Create" button.

One can obtain the Watson OpenScale `instance_id` (guid) by accessing the [cloud console](https://cloud.ibm.com/resources), clicking on `Services` and clicking anywhere on the Watson OpenScale service tile except for the service link and then checking the popping sidebar on the right.

In [ ]:
AIOS_CREDENTIALS = {
    "instance_guid": "*****",
    "apikey": "*****", 
    "url": "https://api.aiopenscale.cloud.ibm.com"
}

Generate or fetch the WML credentials by clicking on `Credentials` in the sidebar of the provisioned WML page and paste it below.

In [ ]:
WML_CREDENTIALS = {
    "apikey": "*****",
    "iam_apikey_description": "*****",
    "iam_apikey_name": "*****",
    "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
    "iam_serviceid_crn": "*****",
    "instance_id": "*****",
    "password": "*****",
    "url": "https://us-south.ml.cloud.ibm.com",
    "username": "*****"
}

<a id="deployment"></a>
## 2. Creating and deploying an image-based model

The dataset used is MNIST dataset of handwritten digits. It consists of 60,000 28x28 grayscale images of the 10 digits, along with a test set of 10,000 images. More information about the dataset can be found here: https://keras.io/datasets/#mnist-database-of-handwritten-digits

Note: Tensorflow versions supported by WML are: 1.2, 1.5, and 1.11. Make sure you have one of these versions before creating the models. Version 1.11 is used in this notebook.

### 2.1 Creating a model

In [ ]:
!pip install keras
!pip install tensorflow==1.11.0
!pip install keras_sequential_ascii

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras_sequential_ascii import sequential_model_to_ascii_printout
from keras import backend as keras_backend
print(keras.__version__)

In [ ]:
batch_size = 128
num_classes = 10
epochs = 5

In [ ]:
# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if keras_backend.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

In [ ]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [ ]:
# Define Model

def base_model():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.Adadelta(),
                  metrics=['accuracy'])
    return model

In [ ]:
cnn_n = base_model()
cnn_n.summary()

In [ ]:
# Vizualizing model structure
sequential_model_to_ascii_printout(cnn_n)

In [ ]:
# Fit model
print(y_train.shape)
cnn = cnn_n.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_test, y_test))

In [ ]:
scores = cnn_n.evaluate(x_test, y_test, verbose=0)
print(scores)
print("Accuracy: %.2f%%" % (scores[1]*100))

### 2.2 Storing the model

In [ ]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

wml_client = WatsonMachineLearningAPIClient(WML_CREDENTIALS)
cnn_n.save("mnist_cnn.h5")
!rm mnist_cnn.tar*
!tar -czvf mnist_cnn.tar.gz mnist_cnn.h5

In [ ]:
!rm mnist_cnn.h5

In [ ]:
model_name = "MNIST Model"

# Update the FRAMEWORK_VERSION below depending on the tensorflow version used
model_meta = {
    wml_client.repository.ModelMetaNames.NAME: model_name,
    wml_client.repository.ModelMetaNames.DESCRIPTION: "MNIST model",
    wml_client.repository.ModelMetaNames.FRAMEWORK_NAME: "tensorflow",
    wml_client.repository.ModelMetaNames.FRAMEWORK_VERSION: "1.11",
    wml_client.repository.ModelMetaNames.FRAMEWORK_LIBRARIES: [
         {"name": "keras", "version": "2.2.4"}
    ]
}

In [ ]:
published_model_details = wml_client.repository.store_model(model='mnist_cnn.tar.gz', meta_props=model_meta)

In [ ]:
model_uid = wml_client.repository.get_model_uid(published_model_details)
model_uid

### 2.3 Deploying the model

In [ ]:
deployment= wml_client.deployments.create(name= model_name + " Deployment", model_uid=model_uid)

In [ ]:
scoring_url = wml_client.deployments.get_scoring_url(deployment)
print(scoring_url)

## 3. Subscriptions

### 3.1 Configuring OpenScale

In [ ]:
from ibm_ai_openscale import APIClient
from ibm_ai_openscale.engines import WatsonMachineLearningAsset

aios_client = APIClient(AIOS_CREDENTIALS)
aios_client.version

### 3.2 Subscribe the asset

In [ ]:
from ibm_ai_openscale.supporting_classes import *

subscription = aios_client.data_mart.subscriptions.add(WatsonMachineLearningAsset(
    model_uid,
    problem_type=ProblemType.MULTICLASS_CLASSIFICATION,
    input_data_type=InputDataType.UNSTRUCTURED_IMAGE,
    probability_column='probability'
))

In [ ]:
aios_client.data_mart.subscriptions.list()

In [ ]:
subscription.get_details()

### 3.3 Score the model and get transaction-id

In [ ]:
!pip install numpy
!pip install matplotlib

import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline 
img = np.array(x_test[999], dtype='float')
pixels = img.reshape((28, 28))
plt.imshow(pixels, cmap='gray')
plt.show()

In [ ]:
scoring_data = {'values': [x_test[999].tolist()]}
predictions = wml_client.deployments.score(scoring_url, scoring_data)
print(predictions)

In [ ]:
transaction_id = subscription.payload_logging.get_table_content().scoring_id[0]

<a id="explainability"></a>
## 4. Explainability

### 4.1 Configure Explainability

In [ ]:
subscription.explainability.enable()

In [ ]:
subscription.explainability.get_details()

### 4.2 Get explanation for the transaction

In [ ]:
explanation = subscription.explainability.run(transaction_id, background_mode=False)

In [ ]:
explanation

### The explanation images can be obtained using the cells below

In [ ]:
!pip install Pillow
from PIL import Image
import base64
import io

img = explanation["entity"]["predictions"][0]["explanation_features"][0]["full_image"]
img_data = base64.b64decode(img)
Image.open(io.BytesIO(img_data))

In [ ]:
img = explanation["entity"]["predictions"][1]["explanation_features"][0]["full_image"]
img_data = base64.b64decode(img)
Image.open(io.BytesIO(img_data))